In [2]:
import torch
from torch.multiprocessing import Pool, Process, set_start_method

In [3]:
u1 = torch.tensor([2, 3, 3])
v1 = torch.tensor([6, 4, 6])
result1 = torch.stack((u1, v1), dim=1)

u2 = torch.tensor([3, 6, 1])
v2 = torch.tensor([6, 2, 4])
result2 = torch.stack((u2, v2), dim=1)

print(result1.shape, result2.shape)
# Check if all UV values in result1 are in result2
mask = torch.isin(result1, result2).all(dim=1)
print(mask)

torch.Size([3, 2]) torch.Size([3, 2])
tensor([True, True, True])


In [4]:
t = torch.tensor([[1, 0, 3], 
                  [4, 0, 2],
                  [4, 0, 2]])
print(t.shape)
mask = torch.all(t == 0, dim=1, keepdim=True)
print(mask, mask.shape)

torch.Size([3, 3])
tensor([[False],
        [False],
        [False]]) torch.Size([3, 1])


In [4]:
t = torch.tensor([
    [
        [1, 0, 3],
        [4, 5, 2],
        [4, 0, 2],
        [4, 0, 2]

    ],
    [
        [1, 0, 3],
        [4, 5, 2],
        [4, 0, 2],
        [4, 0, 2]
    ],
    [
        [1, 0, 3],
        [4, 5, 2],
        [4, 0, 2],
        [4, 0, 2]

    ],
    [
        [1, 0, 3],
        [4, 5, 2],
        [4, 0, 2],
        [4, 0, 2]
    ]
])
print(t.shape)
mask = torch.any(t == 0, dim=2, keepdim=True)
# print(mask, mask.shape)

torch.Size([4, 4, 3])


In [5]:
uv = [[0,1], [1,2], [2,3], [3,3]]
uv = torch.tensor(uv)

print(uv.shape)

rgb = [[0,1,2], [1,2,3], [2,3,4], [3,4,5]]
rgb = torch.tensor(rgb)

t[uv[:,0], uv[:,1]]

torch.Size([4, 2])


tensor([[4, 5, 2],
        [4, 0, 2],
        [4, 0, 2],
        [4, 0, 2]])

In [ ]:
        # showRasterizedImageComp(current_raster, global_raster)
        # cv2.imwrite("Current.png", (current_raster* 255).astype(np.uint8))
        # cv2.imwrite("Global.png", (global_raster* 255).astype(np.uint8))
        
        # # print(indices.shape, global_pixel_indices.shape)
        # # Find indices of filtered_points that are not present in global_pixel_indices
        # # not_in_global = torch.isin(indices, global_pixel_indices).all(dim=1)
        # not_in_global = ~customIsIn2Batched(indices, global_pixel_indices)
        
        # # Get filtered_points that are not present in global_filtered_points
        # filtered_points_not_in_global = filtered_points[not_in_global]
        # filtered_colors_not_in_global = filtered_colors[not_in_global]
        # indices_not_in_global = indices[not_in_global]

        # showFilterMaskImage(indices_not_in_global)
        # raster = showRasterizedImage(indices_not_in_global[:,0], indices_not_in_global[:,1], filtered_colors_not_in_global)
        # plt.imshow(raster)
        # plt.show()


        # print("Adding new points: ", filtered_points_not_in_global.shape)
        
        # saveTensorAsPLY(filtered_points_not_in_global,filtered_colors_not_in_global, filtered_ply_file_name)
        # Append to global points
        # global_points = torch.cat((global_points, filtered_points_not_in_global), dim=0)
        # global_colors = torch.cat((global_colors, filtered_colors_not_in_global), dim=0)

# saveTensorAsPLY(global_points,global_colors, "Global.ply")

In [ ]:
def customIsIn(result1, result2):
    mask = []
    for i in range(result1.shape[0]):
        for UV in result2:
            if torch.equal(UV, result1[i]):
                mask.append(True)
                break
        if len(mask) != i+1:
            mask.append(False)
    return torch.tensor(mask, device=result1.device)

mask1 = customIsIn(result1, result2)
print(mask1)


def customIsIn2(result1, result2):
    result1_expanded = result1.unsqueeze(1)
    result2_expanded = result2.unsqueeze(0)
    is_combination_in_result2 = torch.all(result1_expanded == result2_expanded, dim=2)
    # print(is_combination_in_result2)
    return torch.any(is_combination_in_result2, dim=1)

def customIsIn2Batched(result1, result2):
    # Set a batch size based on your available GPU memory
    batch_size = 1000

    # Number of batches
    num_batches = result1.shape[0] // batch_size + 1

    # Initialize an empty tensor to store the comparison results
    is_combination_in_result2 = torch.empty(result1.shape[0], result2.shape[0], dtype=torch.bool)

    # Process in batches
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, result1.shape[0])

        # Extract a batch from result1
        result1_batch = result1[start_idx:end_idx].unsqueeze(1)

        # Expand dimensions for broadcasting
        result1_batch_expanded = result1_batch.unsqueeze(2)
        result2_expanded = result2.unsqueeze(0).unsqueeze(0)

        # Element-wise comparison for the batch
        is_combination_in_result2[start_idx:end_idx] = torch.all(result1_batch_expanded == result2_expanded, dim=-1)

    return torch.any(is_combination_in_result2, dim=2)

mask2 = customIsIn2(result1, result2)
print(mask2)
